# Sending
## Example 4. Pub/Sub with Direct exchange using routing key.
- Sender generates logs with types 'error', 'info', 'warning'
- First Receiver accepts all of log types.
- Second Receiver accepts only 'error' log type.

### Routing

In the previous tutorial we built a simple logging system. We were able to broadcast log messages to many receivers.

In this tutorial we're going to add a feature to it - we're going to make it possible to subscribe only to a subset of the messages. For example, we will be able to direct only critical error messages to the log file (to save disk space), while still being able to print all of the log messages on the console.


In [4]:
import pika

EXCHANGE_NAME = 'direct_logs'

connection = pika.BlockingConnection(pika.ConnectionParameters(
        host='192.168.99.100'))
channel = connection.channel()

### Direct exchange

Our logging system from the previous tutorial broadcasts all messages to all consumers. We want to extend that to allow filtering messages based on their severity. For example we may want the script which is writing log messages to the disk to only receive critical errors, and not waste disk space on warning or info log messages.

We were using a fanout exchange, which doesn't give us too much flexibility - it's only capable of mindless broadcasting.

We will use a direct exchange instead. The routing algorithm behind a direct exchange is simple - a message goes to the queues whose binding key exactly matches the routing key of the message.

To illustrate that, consider the following setup:
![](https://www.rabbitmq.com/img/tutorials/direct-exchange.png)

In this setup, we can see the direct exchange X with two queues bound to it. The first queue is bound with binding key orange, and the second has two bindings, one with binding key black and the other one with green.

In such a setup a message published to the exchange with a routing key orange will be routed to queue Q1. Messages with a routing key of black or green will go to Q2. All other messages will be discarded.

#### Multiple bindings
![](https://www.rabbitmq.com/img/tutorials/direct-exchange-multiple.png)
It is perfectly legal to bind multiple queues with the same binding key. In our example we could add a binding between X and Q1 with binding key black. In that case, the direct exchange will behave like fanout and will broadcast the message to all the matching queues. A message with routing key black will be delivered to both Q1 and Q2.



## Emitting logs
We'll use this model for our logging system. Instead of fanout we'll send messages to a direct exchange. We will supply the log severity as a routing key. That way the receiving script will be able to select the severity it wants to receive. Let's focus on emitting logs first.

Like always we need to create an exchange first:

In [5]:
channel.exchange_declare(exchange=EXCHANGE_NAME,
                         type='direct')

<METHOD(['channel_number=1', 'frame_type=1', 'method=<Exchange.DeclareOk>'])>

![](https://www.rabbitmq.com/img/tutorials/python-four.png)

And we're ready to send a message:

In [6]:
# To simplify things we will assume that 'severity'
# can be one of 'info', 'warning', 'error'.

ERROR_MSG = 'error'
INFO_MSG = 'info'
WARNING_MSG = 'warning'


def send_msg(severity, message):
    channel.basic_publish(exchange=EXCHANGE_NAME,
                          routing_key=severity,
                          body=message)
    print " [x] Sent %r:%r" % (severity, message)

 [x] Sent 'error':'Hello World!'


In [11]:
send_msg(ERROR_MSG, 'Hello World!')
send_msg(INFO_MSG, 'Hello World!')
send_msg(INFO_MSG, 'Hello World!')
send_msg(WARNING_MSG, 'Hello World!')
send_msg(ERROR_MSG, 'Hello World!')

 [x] Sent 'error':'Hello World!'
 [x] Sent 'info':'Hello World!'
 [x] Sent 'info':'Hello World!'
 [x] Sent 'warning':'Hello World!'
 [x] Sent 'error':'Hello World!'


In [12]:
connection.close()